Upon doing cursory research on the visualization tools and libraries available, I have decided to use R and its libraries to create my visualizations for this project. However, I prefer the functionality of python to R when it comes to data cleaning. So, I will be processing the variables and observations required for the visualization using the pandas library, and exporting the cleaned datasets to an R environment in the form of csv files:

In [1]:
# Required Libraries
import pandas as pd
import copy

Here is the main dataframe containing the data from the simulation:

In [2]:
data = pd.read_csv('simulation.csv')
data.head()

,Turn,Card Draw,Die Roll 1,Die Roll 2,Sum of Dice,Board Position,Doubles,3 Straight Doubles,Space Name
0,1,NaN,1,5,6,0,NaN,NaN,Go
1,2,NaN,2,6,8,6,NaN,NaN,Oriental Ave
2,3,NaN,3,1,4,14,NaN,NaN,Virginia Ave
3,4,NaN,4,2,6,18,NaN,NaN,Tennessee Ave
4,5,NaN,6,4,10,24,NaN,NaN,Illinois Ave


This dataset has 60000 observations and 9 variables.

In [3]:
data.shape

(60000, 9)

In [4]:
# From a dataset called spaces, which shares the 'Space Name' as a key with data, the indices are obtained: 
spaces = pd.read_csv('spaces.csv')
spaces.head()

,Index,Space Name
0,0,Go
1,1,Mediterranean Ave
2,2,Community Chest #1
3,3,Baltic Ave
4,4,Income Tax


In [5]:
# The first visualization requires the counts of each space. So, the dataset will be grouped on space name
viz_1 = data.groupby('Space Name').count()
viz_1.reset_index(inplace=True)

In [6]:
# Then, only the required columns of space name and frequency are extracted
viz_1 = viz_1[['Space Name', 'Turn']].rename(columns = {'Turn': 'Frequency'})
viz_1 = viz_1.join(spaces.set_index('Space Name'), on = 'Space Name')

In [7]:
viz_1.head()

,Space Name,Frequency,Index
0,Atlantic Ave,1491,26
1,B & O Railroad,1671,25
2,Baltic Ave,1225,3
3,Boardwalk,1443,39
4,Chance #1,1316,7


In [8]:
# The monopoly board is 11 x 11 with 40 spaces. So, we need to extend the grid and fill it to contain the required 121 entries.
row_structure = ['Not', 800]
rows = []
for i in range(81):
    mid = copy.deepcopy(row_structure)
    mid.append(40 + i)
    rows.append(mid)
dummy = pd.DataFrame(rows, columns = ['Space Name', 'Frequency', 'Index'])
dummy.head()

,Space Name,Frequency,Index
0,Not,800,40
1,Not,800,41
2,Not,800,42
3,Not,800,43
4,Not,800,44


In [9]:
v1 = pd.concat([viz_1, dummy], ignore_index = True)
v1

,Space Name,Frequency,Index
0,Atlantic Ave,1491,26
1,B & O Railroad,1671,25
2,Baltic Ave,1225,3
3,Boardwalk,1443,39
4,Chance #1,1316,7
...,...,...,...
116,Not,800,116
117,Not,800,117
118,Not,800,118
119,Not,800,119


In [10]:
# Now we need to fill the x and y coordinates (Only borders contain actual game spaces)
def x_fill(index):
    if 0 <= index <= 10:
        return 0
    elif 11 <= index <= 19:
        return index % 10
    elif 20 <= index <= 30:
        return 10
    elif 31 <= index <= 39:
        return 40 - index
    elif index >= 40:
        val = index - 40
        return (val % 9) + 1

In [11]:
def y_fill(index):
    if 0 <= index <= 10:
        return index
    elif 11 <= index <= 20:
        return 10
    elif 21 <= index <= 29:
        return 30 - index
    elif 30 <= index <= 39:
        return 0
    elif index >= 40:
        val = index - 40
        return (val // 9) + 1

In [12]:
x_val = v1['Index'].apply(x_fill)
y_val = v1['Index'].apply(y_fill)

In [13]:
v1_f = v1.assign(x = x_val, y = y_val)

In [14]:
v1_f

,Space Name,Frequency,Index,x,y
0,Atlantic Ave,1491,26,10,4
1,B & O Railroad,1671,25,10,5
2,Baltic Ave,1225,3,0,3
3,Boardwalk,1443,39,1,0
4,Chance #1,1316,7,0,7
...,...,...,...,...,...
116,Not,800,116,5,9
117,Not,800,117,6,9
118,Not,800,118,7,9
119,Not,800,119,8,9


In [15]:
v1_f.to_csv('v1.csv', index=False)

The second visualization is to count every turn as an edge, forming a network connecting all the board spaces:

In [16]:
# Overall Board Traversals
next_space = pd.DataFrame(data['Space Name'][1:]).rename(columns = {'Space Name': 'Next'})
next_space = next_space.reset_index()
next_space = next_space[['Next']]

Since the `Space Name` of two consecutive observations represent the move on the previous turn, by simply moving the column up by one space and concatenating both columns, we obtain a list of edges for the network:

In [17]:
next_space = next_space.join(spaces.set_index('Space Name'), on = 'Next')

In [18]:
next_space.insert(0, "Current", data['Space Name'][:59999])

In [19]:
next_space = next_space.rename(columns = {'Index': 'I_next'})

In [20]:
next_space = next_space.join(spaces.set_index('Space Name'), on = 'Current').rename(columns = {'Index': 'I_current'})

The indices are also recorded in case they are required to make the visualization:

In [21]:
next_space

,Current,Next,I_next,I_current
0,Go,Oriental Ave,6,0
1,Oriental Ave,Virginia Ave,14,6
2,Virginia Ave,Tennessee Ave,18,14
3,Tennessee Ave,Illinois Ave,24,18
4,Illinois Ave,Pennsylvania Ave,34,24
...,...,...,...,...
59994,Boardwalk,Jail,10,39
59995,Jail,New York Ave,19,10
59996,New York Ave,Water Works,28,19
59997,Water Works,Park Place,37,28


In [22]:
next_space.to_csv('v2.csv', index=False)

The third process required for a visualization creation is the community chest data, which records the actions within the randomly drawn community chest card. It is also beneficial that every community chest card either results in a profit or loss for the player, and the action is not ambiguous.

In [23]:
community_chest = pd.read_csv('cc.csv')
community_chest.head()

,Card Number,Community Chest Cards,Profit
0,1,Advance to Go (Collect $200),200
1,2,Bank error in your favor – collect $75,75
2,3,Doctor's fees – Pay $50,-50
3,4,Get out of jail free – this card may be kept u...,0
4,5,Go to jail – go directly to jail – Do not pass...,0


To join this dataset onto the main frame, the observations where the player lands on Community Chest are extracted:

In [24]:
turns_req = data[data['Space Name'].str.contains("Community")]
turns_req

,Turn,Card Draw,Die Roll 1,Die Roll 2,Sum of Dice,Board Position,Doubles,3 Straight Doubles,Space Name
8,9,6.0,5,2,7,17,NaN,NaN,Community Chest #2
10,11,5.0,1,6,7,33,NaN,NaN,Community Chest #3
21,22,3.0,3,4,7,2,NaN,NaN,Community Chest #1
38,39,14.0,1,3,4,17,NaN,NaN,Community Chest #2
105,106,16.0,6,2,8,33,NaN,NaN,Community Chest #3
...,...,...,...,...,...,...,...,...,...
59892,59893,1.0,3,5,8,2,NaN,NaN,Community Chest #1
59902,59903,12.0,4,1,5,33,NaN,NaN,Community Chest #3
59924,59925,5.0,1,6,7,17,NaN,NaN,Community Chest #2
59966,59967,4.0,6,1,7,2,NaN,NaN,Community Chest #1


In [25]:
turns_req = turns_req.astype({'Card Draw': 'int64'})

In [26]:
profit = community_chest[['Card Number', 'Profit']]
turns_req = turns_req.join(profit.set_index('Card Number'), on = 'Card Draw')

In [27]:
v3 = turns_req.pivot_table(index = 'Space Name', values = 'Profit', aggfunc = 'sum')
v3.reset_index(inplace = True)

In [28]:
v3

,Space Name,Profit
0,Community Chest #1,32320
1,Community Chest #2,42685
2,Community Chest #3,39405


In [29]:
v3.to_csv('v3.csv', index=False)

The fourth process focuses on the other card drawing mechanism in the game, Chance. Unlike Community Chest, Chance encodes a variety of actions for the player which could include getting money, moving to a specific space on the board or paying other players money. Instead, this visualization focuses on the most likely movement for a player exiting the first Chance spot.

In [30]:
c_turns = data[data['Space Name'].str.contains("Chance #1")]
c_turns = c_turns.join(spaces.set_index('Space Name'), on = 'Space Name')
c_turns

,Turn,Card Draw,Die Roll 1,Die Roll 2,Sum of Dice,Board Position,Doubles,3 Straight Doubles,Space Name,Index
30,31,13.0,1,3,4,7,NaN,NaN,Chance #1,7
50,51,4.0,4,1,5,7,NaN,NaN,Chance #1,7
65,66,13.0,4,6,10,7,NaN,NaN,Chance #1,7
87,88,14.0,2,2,4,7,double,NaN,Chance #1,7
128,129,12.0,1,6,7,7,NaN,NaN,Chance #1,7
...,...,...,...,...,...,...,...,...,...,...
59917,59918,6.0,3,4,7,7,NaN,NaN,Chance #1,7
59945,59946,17.0,6,6,12,7,double,NaN,Chance #1,7
59973,59974,14.0,2,2,4,7,double,NaN,Chance #1,7
59975,59976,5.0,2,4,6,7,NaN,NaN,Chance #1,7


In [31]:
chance = pd.read_csv('chance.csv')
chance = chance[:17]

In [32]:
chance_cols = chance[['Card Number', '7']].astype({'7': 'int64'})

In [33]:
c_turns = c_turns.join(chance_cols.set_index('Card Number'), on = 'Card Draw')
c_turns

,Turn,Card Draw,Die Roll 1,Die Roll 2,Sum of Dice,Board Position,Doubles,3 Straight Doubles,Space Name,Index,7
30,31,13.0,1,3,4,7,NaN,NaN,Chance #1,7,5
50,51,4.0,4,1,5,7,NaN,NaN,Chance #1,7,15
65,66,13.0,4,6,10,7,NaN,NaN,Chance #1,7,5
87,88,14.0,2,2,4,7,double,NaN,Chance #1,7,39
128,129,12.0,1,6,7,7,NaN,NaN,Chance #1,7,7
...,...,...,...,...,...,...,...,...,...,...,...
59917,59918,6.0,3,4,7,7,NaN,NaN,Chance #1,7,11
59945,59946,17.0,6,6,12,7,double,NaN,Chance #1,7,7
59973,59974,14.0,2,2,4,7,double,NaN,Chance #1,7,39
59975,59976,5.0,2,4,6,7,NaN,NaN,Chance #1,7,15


In [34]:
v4 = c_turns.pivot_table(index = '7', values = 'Turn', aggfunc = 'count')
v4.reset_index(inplace = True)
v4 = v4.rename(columns = {'7': 'To', 'Turn': 'Count'})

In [35]:
v4 = v4.join(spaces.set_index('Index'), on = 'To')

In [36]:
v4

,To,Count,Space Name
0,0,84,Go
1,4,81,Income Tax
2,5,78,Reading Railroad
3,7,630,Chance #1
4,10,69,Jail
5,11,85,St. Charles Place
6,15,164,Pennsylvania Railroad
7,24,63,Illinois Ave
8,39,62,Boardwalk


In [37]:
v4.to_csv('v4.csv', index=False)

The Final Visualization will also be a heatmap, but here it will represent and compare the profitability of the different spaces around the board. Here is the metric I chose:

(1 - (Cost of Property / Total Money)) + (Probability of landing on the square) * Rent

In [38]:
#V5
economies = pd.read_csv('economy.csv')
economies.head()

,Space Name,Cost,Rent
0,Jail,0,0
1,Illinois Ave,240,20
2,New York Ave,200,16
3,Go,0,0
4,B & O Railroad,200,25


In [84]:
current = v1_f.join(economies.set_index('Space Name'), on = 'Space Name')

In [85]:
current['Proportion Initial Left'] = 1 - (current['Cost'] / 1500) 

In [86]:
current = current.assign(benefit = current['Proportion Initial Left'] + (current['Frequency'] / 60000) * current['Rent'])

In [87]:
current.sort_values("benefit", ascending = False)[:40]

,Space Name,Frequency,Index,x,y,Cost,Rent,Proportion Initial Left,benefit
3,Boardwalk,1443,39,1,0,400.0,50.0,0.733333,1.935833
1,B & O Railroad,1671,25,10,5,200.0,25.0,0.866667,1.562917
30,Reading Railroad,1618,5,0,5,200.0,25.0,0.866667,1.540833
29,Pennsylvania Railroad,1595,15,5,10,200.0,25.0,0.866667,1.531250
27,Park Place,1241,37,3,0,350.0,35.0,0.766667,1.490583
28,Pennsylvania Ave,1413,34,6,0,320.0,28.0,0.786667,1.446067
26,Pacific Ave,1482,31,9,0,300.0,26.0,0.800000,1.442200
15,Illinois Ave,1746,24,10,6,240.0,20.0,0.840000,1.422000
24,North Carolina Ave,1428,32,8,0,300.0,26.0,0.800000,1.418800
21,Marvin Gardens,1506,29,10,1,280.0,24.0,0.813333,1.415733


In [69]:
v5 = current.fillna(0)

In [71]:
v5.to_csv('v5.csv', index=False)